In [1]:
import csoundengine as ce
import logging
#logging.basicConfig(level="DEBUG")
#ce.logger.setLevel("DEBUG")

In [2]:
e = ce.Engine()
s = e.session()

s.defInstr('foo', r'''
|kfreq=1000, kdetune=1, kgain=0.5|
a0 = oscili(0.1, kfreq)
a0 += oscili(0.1, kfreq+kdetune)
a0 *= kgain
a0 *= cossegr:a(0, 0.05, 1, 0.05, 0)
outch 1, a0
''')

using libsndfile-1.2.2
sr = 48000.0, kr = 750.000, ksmps = 64
0dBFS level = 1.0, A4 tuning = 440.0
audio buffered in 256 sample-frame blocks
reading 2048-byte blks of shorts from adc:Built-in Audio Analog Stereo: (RAW)
writing 512 sample blks of 64-bit floats to dac:Built-in Audio Analog Stereo:
SECTION 1:


Instr(foo, kfreq=1000, kdetune=1, kgain=0.5)

In [3]:
sy = s.sched('foo', args=dict(kfreq=440, kdetune=1))
sy.automate('kdetune', (3, 1, 5, 20), delay=3)

11

In [4]:
sy.stop()

In [5]:
with e:
    gr = ce.synth.SynthGroup([
        s.sched('foo', delay=1, dur=3, args=dict(kfreq=440, kdetune=1)),  # 1 - 4
        s.sched('foo', delay=4.0001, dur=3, args=dict(kfreq=2000, kdetune=5))  # 4 - 8
    ])
    print(gr)
    gr.automate('kdetune', (0, 1, 2, 100), delay=2)
    gr.automate('kdetune', (5, 0, 7, 20), delay=0, overtake=True)
e

SynthGroup(n=2)
    Synth(‖ foo=401.0003 start=6.139 dur=3 kfreq=440 kdetune=1 kgain=0.5)
    Synth(‖ foo=401.0004 start=9.139 dur=3 kfreq=2000 kdetune=5 kgain=0.5)


Engine(name=engine0, sr=48000, backend=jack, outdev=dac:Built-in Audio Analog Stereo:, nchnls=2, indev=adc:Built-in Audio Analog Stereo:, nchnls_i=4, bufferSize=256)

In [6]:
print(gr.synths)
gr.stop()

[Synth(◼ foo=401.0003 start=6.139 dur=3 kfreq=440 kdetune=1 kgain=0.5), Synth(▶ foo=401.0004 start=9.139 dur=3 kfreq=2000 kdetune=5 kgain=0.5)]


In [7]:
gr.synths[0].playStatus()

'stopped'